In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
%matplotlib inline
warnings.filterwarnings("ignore")

In [4]:
train_df = pd.read_csv("Data Sets/train.csv")
test_df = pd.read_csv("Data Sets/test.csv")

In [5]:
train_df.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
2,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1
3,z13lfzdo5vmdi1cm123te5uz2mqig1brz04,ferleck ferles,2013-11-27T21:39:24,Subscribe to my channel ﻿,1
4,z12avveb4xqiirsix04chxviiljryduwxg0,BeBe Burkey,2013-11-28T16:30:13,and u should.d check my channel and tell me wh...,1


In [6]:
test_df.head()

,ID,COMMENT_ID,AUTHOR,DATE,CONTENT
0,0,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...
1,1,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com
2,2,LZQPQhLyRh9-wNRtlZDM90f1k0BrdVdJyN_YsaSwfxc,Jason Haddad,2013-11-26T02:55:11,"Hey, check out my new website!! This site is a..."
3,3,z122wfnzgt30fhubn04cdn3xfx2mxzngsl40k,Bob Kanowski,2013-11-28T12:33:27,i turned it on mute as soon is i came on i jus...
4,4,z13ttt1jcraqexk2o234ghbgzxymz1zzi04,Cony,2013-11-28T16:01:47,You should check my channel for Funny VIDEOS!!﻿


In [7]:
# Cleaning the texts
import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mayur\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
train_df.shape

(1157, 5)

In [10]:
# CLeaning the texts
corpus = []
for i in range(0, 1157):
    review = re.sub("[^a-zA-Z]", " ", train_df["CONTENT"][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words("english"))]
    review = " ".join(review)
    corpus.append(review)

In [13]:
# Feature extraction
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(corpus).toarray()
y = train_df.iloc[:,-1].values

In [15]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [16]:
y

array([1, 1, 1, ..., 0, 0, 0], dtype=int64)

In [17]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

In [18]:
#Apply Kfolds validation on the train data and
#then applying logestic regression
#Accuracy score, confusion matrix, classification report

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import KFold,cross_val_score, cross_val_predict

kfold = KFold(n_splits=10, random_state=None, shuffle=True)

for train, val_index in kfold.split(X):
    #print("Train:", train,"Validation:", val_index)
    X_train, X_test = X[train], X[val_index]
    y_train, y_test = y[train], y[val_index]
    logreg.fit(X_train, y_train)
    #predictions
    Predictions = logreg.predict(X_test)
    print(accuracy_score(y_test, Predictions))
    cm = confusion_matrix(y_test, Predictions)
    print(cm)
    print(classification_report(y_test, Predictions))
    print()
    print("******************************************")

0.9051724137931034
[[49  3]
 [ 8 56]]
              precision    recall  f1-score   support

           0       0.86      0.94      0.90        52
           1       0.95      0.88      0.91        64

    accuracy                           0.91       116
   macro avg       0.90      0.91      0.90       116
weighted avg       0.91      0.91      0.91       116


******************************************
0.9310344827586207
[[56  2]
 [ 6 52]]
              precision    recall  f1-score   support

           0       0.90      0.97      0.93        58
           1       0.96      0.90      0.93        58

    accuracy                           0.93       116
   macro avg       0.93      0.93      0.93       116
weighted avg       0.93      0.93      0.93       116


******************************************
0.9482758620689655
[[56  1]
 [ 5 54]]
              precision    recall  f1-score   support

           0       0.92      0.98      0.95        57
           1       0.98      0.92  

In [19]:
cross_val_score(logreg, X, y)

array([0.86637931, 0.93103448, 0.95238095, 0.94805195, 0.91341991])

In [20]:
cross_val_predict(logreg, X, y)

array([1, 0, 1, ..., 0, 0, 0], dtype=int64)

In [21]:
test_df.shape

(799, 5)

In [23]:
corpus_1 = []
for i in range(0, 799):
    review_1 = re.sub('[^a-zA-Z]', ' ', test_df['CONTENT'][i])
    review_1 = review_1.lower()
    review_1 = review_1.split()
    ps = PorterStemmer()
    review_1 = [ps.stem(word) for word in review_1 if not word in set(stopwords.words('english'))]
    review_1 = ' '.join(review_1)
    corpus_1.append(review_1)

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
c_v = CountVectorizer()
X_1 = c_v.fit_transform(corpus_1).toarray()

In [25]:
Predictions_1 = logreg.predict(X_1)
print(Predictions_1)

ValueError: X has 2050 features per sample; expecting 2484